<a href="https://colab.research.google.com/github/sanket2221/Electrical_AI/blob/master/power_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install pypsa

In [41]:
#!apt-get install -y -qq glpk-utils

In [49]:
from pyomo.opt import SolverFactory
opt = SolverFactory('glpk', executable='/usr/bin/glpsol')

In [83]:
import pypsa
import numpy as np

In [70]:
network = pypsa.Network()
#add three buses
for i in range(3):
    network.add("Bus"," bus {}".format(i),v_nom  = 20.)

In [71]:
print(network.buses)

attribute  v_nom type    x    y  ... v_mag_pu_min  v_mag_pu_max  control  sub_network
 bus 0      20.0       0.0  0.0  ...          0.0           inf       PQ             
 bus 1      20.0       0.0  0.0  ...          0.0           inf       PQ             
 bus 2      20.0       0.0  0.0  ...          0.0           inf       PQ             

[3 rows x 10 columns]


In [72]:
#add three lines in a ring
for i in range(3):
    network.add("Line"," line {}".format(i),
                bus0=" bus {}".format(i),
                bus1=" bus {}".format((i+1)%3),
                x=0.1,
                r=0.01)

In [73]:
print(network.lines)

attribute    bus0    bus1 type    x  ...  b_pu  x_pu_eff  r_pu_eff  s_nom_opt
 line 0     bus 0   bus 1       0.1  ...   0.0       0.0       0.0        0.0
 line 1     bus 1   bus 2       0.1  ...   0.0       0.0       0.0        0.0
 line 2     bus 2   bus 0       0.1  ...   0.0       0.0       0.0        0.0

[3 rows x 26 columns]


In [74]:
#add a generator at bus 0
network.add("Generator"," gen",
            bus=" bus 0",
            p_set=100,
            control="PQ")

In [75]:
print(network.generators)

attribute     bus control  ... ramp_limit_shut_down  p_nom_opt
 gen        bus 0      PQ  ...                  1.0        0.0

[1 rows x 28 columns]


In [76]:
print(network.generators.p_set)

 gen    100.0
Name: p_set, dtype: float64


In [77]:
#add a load at bus 1
network.add("Load"," load",
            bus=" bus 1",
            p_set=100)

In [78]:
print(network.loads)

attribute     bus type  p_set  q_set  sign
 load       bus 1       100.0    0.0  -1.0


In [79]:
print(network.loads.p_set)
network.loads.q_set = 100.

 load    100.0
Name: p_set, dtype: float64


In [80]:
#Do a Newton-Raphson power flow
network.pf()

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object')
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.048747 seconds


{'converged':         0
 now  True, 'error':                 0
 now  4.753531e-10, 'n_iter':      0
 now  3}

In [81]:
print(network.lines_t.p0)

        line 0     line 1     line 2
now  66.897487 -33.333333 -33.391038


In [84]:
print(network.buses_t.v_ang*180/np.pi)

      bus 0     bus 1     bus 2
now     0.0 -0.875939 -0.433813


In [85]:
print(network.buses_t.v_mag_pu)

      bus 0     bus 1    bus 2
now     1.0  0.981199  0.99057
